<a href="https://colab.research.google.com/github/ramscodehub/ai-bot/blob/master/chatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#try:
  #%tensorflow_version 2.x
#except Exception:
  #pass
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras as keras  
print(tf.__version__)
from google.colab import drive
drive.mount("/content/drive")
#!ls "/content/drive/My Drive/"
lines=open('/content/drive/My Drive/movie_lines.txt',encoding='utf-8',errors='ignore').read().split('\n')
conv_lines=open('/content/drive/My Drive/movie_conversations.txt',encoding='utf-8',errors='ignore').read().split('\n')
print(type(conv_lines))
print(type(lines))

1.15.0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'list'>
<class 'list'>


Using TensorFlow backend.


In [ ]:
lines[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

In [ ]:
conv_lines[0:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

In [ ]:
#creating a dictionary to map each lines id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

print(len(id2line))
print(id2line['L1045'])
print(id2line['L1044'])
#Accessing last key in a id2line dictionary
a=list(id2line.keys())
print(a[-1])

304713
They do not!
They do to!
L666256


In [ ]:
id2line

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

In [ ]:
#Accessing indexes of lines in movie_conversations.txt
convos=[]
for line in conv_lines:
  _line=line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
  convos.append(_line.split(","))
  #convos.append(_line)

In [ ]:
convos[:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

In [ ]:
#Arranging the sentences into questions and answers
questions=[]
answers=[]
for conv in  convos:
  for i in range(len(conv)-1):
    questions.append(id2line[conv[i]])
    answers.append(id2line[conv[i+1]])

#Printing the loaded questions and answers sets

for i in range(0,10):
  print(questions[i])
  print(answers[i])
  print()

print(len(questions))
print(len(answers))


Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?
Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.

Cameron.
The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.

The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.
Seems like she could get a date easy enough...

Why?
Unsolved mystery.  She used to be really popular when she started h

In [ ]:
import re
def cleanText(text):
  text=text.lower()
  text=re.sub(r"'bout","about",text)
  text=re.sub(r"that's","that is",text)
  text=re.sub(r"she's","she is",text)
  text=re.sub(r"he's","he is",text)
  text=re.sub(r"where's","where is",text)
  text=re.sub(r"\'ve"," have",text)
  text=re.sub(r"\'d"," would",text)
  text=re.sub(r"\n't","not",text)
  text=re.sub(r"won't","will not",text)
  text=re.sub("didn't","did not",text)
  text=re.sub("don't","do not",text)
  text=re.sub("it's","it is",text)
  text=re.sub("can't","cannot",text)
  text=re.sub(r"you're","you are",text)
  text=re.sub(r"\'re"," are",text)
  text=re.sub(r"[-(),:/;.?]","",text)
  text=re.sub(r"what's","what is",text)
  text=re.sub("there's","there is",text)
  text=re.sub(r"\'ve","have",text)
  text=re.sub(r"i'm","i am",text)
  return text

In [ ]:
#questions and answers after applying changes
cl_questions=[]
cl_answers=[]
for question in questions:
  cl_questions.append(cleanText(question))
for answer in answers:
  cl_answers.append(cleanText(answer)) 

for i in range(0,10):
  print(cl_questions[i])
  print(cl_answers[i])
  print()

print(len(cl_questions))
print(len(cl_answers))

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
well i thought we would start with pronunciation if that is okay with you

well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please

not the hacking and gagging and spitting part  please
okay then how about we try out some french cuisine  saturday  night

you are asking me out  that is so cute what is your name again
forget it

no no it is my fault  we did not have a proper introduction 
cameron

cameron
the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does

the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does
seems like she could get a date easy enough

why
unsolved mystery  she used to be really popular when she started high school then it was just lik

In [ ]:
cl_questions[:11]
#cl_questions[-1]

['can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again',
 'well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'you are asking me out  that is so cute what is your name again',
 'no no it is my fault  we did not have a proper introduction ',
 'cameron',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does',
 'why',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
 'gosh if only we could find kat a boyfriend',
 "c'esc ma tete this is my head"]

In [ ]:
cl_answers[:10]

['well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'okay then how about we try out some french cuisine  saturday  night',
 'forget it',
 'cameron',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does',
 'seems like she could get a date easy enough',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
 'that is a shame',
 'let me see what i can do']

In [ ]:
leng=[]
for i in cl_questions:
  leng.append(len(i.split()))
for j in cl_answers:
  leng.append(len(j.split()))
print(len(leng))
print(leng[0])
print(leng[len(cl_questions)-1])
lengDf=pd.DataFrame(leng,columns=["counts"])
lengDf.head()

443232
22
22


,counts
0,22
1,14
2,9
3,14
4,13


In [ ]:
cl_questions[:5]

['can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again',
 'well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'you are asking me out  that is so cute what is your name again',
 'no no it is my fault  we did not have a proper introduction ']

In [ ]:
#Filtering out the questions that are too short or that are too long
min_length=2
max_length=20

#filtering questions
sorted_questions_q=[]
sorted_answers_q=[]
i=0
for question in cl_questions:
  if(len(question.split())>=min_length and len(question.split())<=max_length):
    sorted_questions_q.append(question)
    sorted_answers_q.append(cl_answers[i])
  i=i+1

#filtering answers from sorted questions
sorted_questions=[]
sorted_answers=[]
i=0
for answer in sorted_answers_q:
  if(len(answer.split())>=min_length and len(answer.split())<=max_length):
      sorted_answers.append(answer)
      sorted_questions.append(sorted_questions_q[i])
  i=i+1

print(len(sorted_questions))
print(len(sorted_answers))


139076
139076


In [ ]:
#Checking the frequency of words
vocab={}
for question in sorted_questions:
  for word in question.split():
    if word not in vocab:
      vocab[word]=1
    else:
      vocab[word]=vocab[word]+1
for answer in sorted_answers:
  for word in answer.split():
    if word not in vocab:
      vocab[word]=1
    else:
      vocab[word]=vocab[word]+1

a=vocab["hacking"]
print(a)
print(len(vocab))

7
53366


In [ ]:
sorted_questions[:10]

['well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'you are asking me out  that is so cute what is your name again',
 'gosh if only we could find kat a boyfriend',
 "c'esc ma tete this is my head",
 'that is because it is such a nice one',
 'how is our little find the wench a date plan progressing',
 'you have my word  as a gentleman',
 'how do you get your hair to look like that',
 'sure have']

In [ ]:
sorted_answers[:10]

['not the hacking and gagging and spitting part  please',
 'okay then how about we try out some french cuisine  saturday  night',
 'forget it',
 'let me see what i can do',
 'right  see  you are ready for the quiz',
 'forget french',
 'well there is someone i think might be ',
 'you are sweet',
 "eber's deep conditioner every two days and i never ever use a blowdryer without the diffuser attachment",
 'i really really really wanna go but i cannot  not unless my sister goes']

In [ ]:
print(dict((list(vocab.items()))[:10]))

{'well': 7445, 'i': 94965, 'thought': 2323, 'we': 18084, 'would': 9056, 'start': 711, 'with': 10258, 'pronunciation': 1, 'if': 7400, 'that': 30959}


In [ ]:
#creating dictionary for mapping each word to a unique number
questions_vocab_int={}
threshold=10
word_number=0
for word,count in vocab.items():
  if(count>=threshold):
    questions_vocab_int[word]=word_number
    word_number += 1

answers_vocab_int={}
threshold=10
word_number=0
for word,count in vocab.items():
  if(count>=threshold):
    answers_vocab_int[word]=word_number
    word_number += 1
print(len(vocab))
print(len(questions_vocab_int))
print(len(answers_vocab_int))
print(dict(list(questions_vocab_int.items())[:20]))
print(dict(list(answers_vocab_int.items())[:20]))

53366
8400
8400
{'well': 0, 'i': 1, 'thought': 2, 'we': 3, 'would': 4, 'start': 5, 'with': 6, 'if': 7, 'that': 8, 'is': 9, 'okay': 10, 'you': 11, 'not': 12, 'the': 13, 'and': 14, 'spitting': 15, 'part': 16, 'please': 17, 'are': 18, 'asking': 19}
{'well': 0, 'i': 1, 'thought': 2, 'we': 3, 'would': 4, 'start': 5, 'with': 6, 'if': 7, 'that': 8, 'is': 9, 'okay': 10, 'you': 11, 'not': 12, 'the': 13, 'and': 14, 'spitting': 15, 'part': 16, 'please': 17, 'are': 18, 'asking': 19}


In [ ]:
#checking for the unique number of last vocabulary
dict(list(questions_vocab_int.items())[::len(list(questions_vocab_int.items()))-1])

{'ext': 8399, 'well': 0}

In [ ]:
#adding unique numbers to start,end and unknown words in a vocabulary
codes=['<PAD>','<EOS>','<GO>','<UNK>']
for code in codes:
  questions_vocab_int[code]=len(questions_vocab_int)+1
for code in codes:
  answers_vocab_int[code]=len(questions_vocab_int)+1

In [ ]:
#Creating dictionary to map unique numbers to their respective  words
questions_int_vocab={}
answers_int_vocab={}
for x,y in questions_vocab_int.items():
  questions_int_vocab[y]=x
dict(list(questions_int_vocab.items())[::len(list(questions_int_vocab.items()))-1])
questions_vocab_int['<PAD>']
#print(len(questions_int_vocab))

8401

In [ ]:
for i in range(len(sorted_answers)):
  sorted_answers[i]=sorted_answers[i] + ' <EOS>'
for i in range(3):
  print(sorted_answers[i])

not the hacking and gagging and spitting part  please <EOS>
okay then how about we try out some french cuisine  saturday  night <EOS>
forget it <EOS>


In [ ]:
sorted_questions[:3]

['well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'you are asking me out  that is so cute what is your name again']

In [ ]:
#converting a total Questions to a set of unique numbers
question_int=[]
for question in sorted_questions:
  ints=[]
  for word in question.split():
    if word not in questions_vocab_int:
      ints.append(questions_vocab_int['<UNK>'])
    else:
      ints.append(questions_vocab_int[word])
  question_int.append(ints)
#Converting total answers to a set of unique numbers
answer_int=[]
for answer in sorted_answers:
  ints=[]
  for word in answer.split():
    if word not in questions_vocab_int:
      ints.append(questions_vocab_int['<UNK>'])
    else:
      ints.append(questions_vocab_int[word])
  answer_int.append(ints)
print(sorted_questions[2])
print(sorted_answers[0])
print(question_int[2])
print(answer_int[0])
print(len(question_int))
print(len(answer_int))

you are asking me out  that is so cute what is your name again
not the hacking and gagging and spitting part  please <EOS>
[11, 18, 19, 20, 21, 8, 9, 22, 23, 24, 9, 25, 26, 27]
[12, 13, 8404, 14, 8404, 14, 15, 16, 17, 8402]
139076
139076


In [ ]:
#Sorting questions and answers wrt to length of questions
actual_questions=[]
actual_answers=[]

for length in range(1,max_length+1):
  for j in enumerate(question_int):
    if len(j[1])==length:
      actual_questions.append(question_int[j[0]])
      actual_answers.append(answer_int[j[0]])
      
print(len(actual_questions))
print(len(actual_answers))
print()
for i in range(5):
  print(actual_questions[i])
  print(actual_answers[i])
  print()


139076
139076

[59, 49]
[1, 60, 60, 60, 61, 62, 63, 1, 64, 12, 65, 37, 66, 67, 8402]

[1, 84]
[11, 193, 174, 56, 62, 21, 6, 8404, 158, 11, 8402]

[0, 85]
[152, 8, 9, 164, 11, 484, 56, 277, 8402]

[24, 87]
[20, 36, 8404, 88, 8404, 1, 89, 58, 90, 91, 8402]

[71, 10]
[1, 1010, 22, 8402]



In [ ]:
#tf.strided_slice(list,begin,end,stride) function demonistration
t = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]])
print(t[2][1])
print(tf.strided_slice(t, [1, 0, 0], [2, 2, 4], [1, 1, 1]))
print(tf.strided_slice(t, [1, 0, 0], [2, 2, 3], [1, 1, 1]))                                                   
print(tf.strided_slice(t, [1, -1, 0], [2, -3, 3], [1, -1, 1]))
b=tf.constant([[1,2,3,4],
               [8,5,6,7]])
print(b[0][3])
print(b[1][0])
print(tf.strided_slice(b, [0,0], [2, -1], [1,1]))

Tensor("strided_slice_1:0", shape=(3,), dtype=int32)
Tensor("StridedSlice:0", shape=(1, 2, 3), dtype=int32)
Tensor("StridedSlice_1:0", shape=(1, 2, 3), dtype=int32)
Tensor("StridedSlice_2:0", shape=(1, 2, 3), dtype=int32)
Tensor("strided_slice_3:0", shape=(), dtype=int32)
Tensor("strided_slice_5:0", shape=(), dtype=int32)
Tensor("StridedSlice_3:0", shape=(2, 3), dtype=int32)


In [ ]:
# Creating placeholders for the inputs and the targets
def model_inputs():
    inputs = tf.compat.v1.placeholder(tf.int32, [None, None], name = 'input')
    targets = tf.compat.v1.placeholder(tf.int32, [None, None], name = 'target')
    lr = tf.compat.v1.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.compat.v1.placeholder(tf.float32, name = 'keep_prob')
    return inputs, targets, lr, keep_prob

In [ ]:
#Removing the last word id from each sentence and conacat <GO> to beginning of each sentence
# Preprocessing the targets
def preprocess_targets(targets, questions_vocab_int, batch_size):
    left_side = tf.fill([batch_size, 1], questions_vocab_int['<GO>'])
    right_side = tf.strided_slice(targets, [0,0], [batch_size, -1], [1,1])
    preprocessed_targets = tf.concat([left_side, right_side], 1)
    return preprocessed_targets

In [ ]:
#Padding all answers to length of largest answer
#not a useful function
def padding_actual_answers(actual_answers,questions_vocab_int):
  actual_answers_arr=np.array(actual_answers)                  
  max_l = max(len(row) for row in actual_answers_arr)
  actual_padded = np.array([row + [questions_vocab_int['<PAD>']] * (max_l - len(row)) for row in actual_answers_arr])
  actual_answers_tensor = tf.convert_to_tensor(actual_padded)
  return actual_answers_tensor

In [ ]:
# Creating the Encoder RNN
def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
    encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                                    cell_bw = encoder_cell,
                                                                    sequence_length = sequence_length,
                                                                    inputs = rnn_inputs,
                                                                    dtype = tf.float32)
    return encoder_state

In [ ]:
#encoder_state---> rnn_decoder gets encoder state as the input
#decoder_cell--->rnn_cells in the decoder
#deocder_embedded_inputs--->mapping of words to vectors of real numbers
#output_function--->used to return output of decoding
#keep_probability--->for applying dropout
#batch_size--->size of each batch
#The purpose of the embeddings matrix is to compute more efficiently the embedding input.
#Basically, you multiply your vector of inputs by the embeddings matrix to get your embedded inputs  

#attention_states-->hidden states to attend over
#num_units-->hiddenStates dimensions
#attention_option: how to compute attention, either "luong" or "bahdanau".

# Decoding the training set
def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              name = "attn_dec_train")
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                              training_decoder_function,
                                                                                                              decoder_embedded_input,
                                                                                                              sequence_length,
                                                                                                              scope = decoding_scope)
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)






In [ ]:
#Model Yet to be built

In [ ]:
#Create the decoder RNN

In [ ]:
#Have to built the sequence to sequence model

In [ ]:
#Training and Testing